In [1]:
from owlready2 import *
#from owlready.editor import *
#from owlready.instance_editor import *
import types
import rdflib
from pprint import pprint
import editobj3
from bs4 import BeautifulSoup
import json

In [2]:
onto = get_ontology("http://test.org/onto.owl")

In [3]:
def string_cleanup(s):
    s = s.strip()
    problem_chars = ['"', '|', "'", '-', ',', '.', '–']
    for i in problem_chars:
        if i in s:
            s = s.replace(i, '')
    s = s.replace(" ", "_")
    s = s.replace(' ', "_")
    s = s.replace('#', "_")
    s = s.replace("&", "and")
    s = s.replace("+", "plus")
    s = s.replace("%", "percent")
    return s
    
def retrieve_info_from_page(asin):

    item_characteristics_dict = {}
    try:
        api_results = json.load(open('./product_data/' + asin + '.json', 'r'))
        
        for key in api_results:
            if type(api_results[key]) is str:
                item_characteristics_dict[key] = string_cleanup(api_results[key])
            else:
                item_characteristics_dict[key] = api_results[key]
            
    except Exception as e: 
        print(e)
        print(asin)

    if asin+'.html' in os.listdir('./pages'):
        
        print('scraping ' + asin) 
        
        page = open('./pages/'+asin+".html", 'r')
        soup = BeautifulSoup(page.read(), 'html.parser')
        tab = soup.find('div', {'id': 'HLCXComparisonWidget_feature_div'})

        # PRODUCT TITLE
        title = soup.find('span', {'id':'productTitle'})
        if title != None:        
            title = string_cleanup(title.text)
            item_characteristics_dict['title'] = title
        else: 
            print("this asin: " + asin + " has problems on title")

        # PRODUCT ASIN
        item_characteristics_dict['asin'] = string_cleanup(asin)

        # PRODUCT CLASSES
        item_characteristics_dict['classes'] = []
        tmp = soup.find('div', {'id':'wayfinding-breadcrumbs_feature_div'})
        if tmp!=None:
            for i in tmp.findAll('a'):
                item_characteristics_dict['classes'].append(string_cleanup(i.text))
        else:
            print("this asin: " + asin + " has problems on classes")

        # PRODUCT BRAND
        if soup.find('a', {'id': 'bylineInfo'}) != None:
            item_characteristics_dict['brand'] = string_cleanup(soup.find('a', {'id': 'bylineInfo'}).text)
        elif soup.find('a', { 'id' : 'brand' }) != None:
            item_characteristics_dict['brand'] = string_cleanup(soup.find('a', { 'id' : 'brand' })['href'].split('/')[1])

        bullets = soup.find('div', {'id':'feature-bullets'})
        #for span in bullets.findAll('span'):
            #print span.text.strip()

        try:
            
            if tab.findAll('th', {'class' : 'comparison_attribute_name_column comparison_table_first_col'}):
                for i, tr in enumerate(tab.findAll('tr')):
                    if i==0: continue
                    th = tr.find('th')
                    td = tr.find('td')
                    if th != None: 
                        if th.text.strip() == "Top Customer Reviews": continue
                        item_characteristics_dict[string_cleanup(th.text.lower())] = string_cleanup(td.text.lower())
                

            else:
                for i, tr in enumerate(tab.findAll('tr')):

                    # tr 0:
                    # tr 1:
                    # tr 2:
                    # tr 3: customer rating
                    if i<4: continue

                    key = tr.findAll('td')[0].span.text
                    if tr.findAll('td')[1].span != None: 
                        value = tr.findAll('td')[1].span.text
                    else: 
                        value = tr.findAll('td')[1].a.text
                    if key == "Price": 
                        value = value.strip()
                        item_characteristics_dict[key.lower()] = float(value.split('$')[1])   
                    elif '$' not in key: 
                        item_characteristics_dict[string_cleanup(key.lower())] = string_cleanup(value.lower())
        except: 
            print('no tr tab')

        # PROD DETAILS
        details_tab = soup.find('div', {'id':'prodDetails'})

        if soup.find('div', {'id':'productDescription'})!=None: 
            description1 = soup.find('div', {'id':'productDescription'}).p.text
    
    return item_characteristics_dict

In [4]:
dict_list = []
for i, page in enumerate(os.listdir('./product_data')):
    dict_list.append(retrieve_info_from_page(page[:-5]))
    
json.dump(dict_list, open('tmp.json', 'w'))

scraping B00QHYI488
scraping B00R25VZGO
Expecting value: line 37 column 17 (char 2390)
B01MUSD2ST
scraping B01MUSD2ST
scraping B00508I446
scraping B00412KHK6
no tr tab
scraping B00I92AUOW
scraping B01G1NHCAA
scraping B01MRZFBBH
scraping B01MR364YB
this asin: B01MR364YB has problems on title
scraping B01G7JCEE2
scraping B0186S41XE
scraping B00WHE2WCG
scraping B01M24QYPN
scraping B015FZLA8A
Expecting value: line 2 column 21 (char 22)
B06XF2LGTM
scraping B06XF2LGTM
no tr tab
scraping B005VIWMCU
scraping B0736R3W1F
scraping B016CJRHR6
this asin: B016CJRHR6 has problems on classes
no tr tab
scraping B01LQV73J2
scraping B00TCVHGZU
scraping B00NACK1DG
scraping B004UR8628
scraping B00YT13T3Q
scraping B01N4VMLGA
scraping B072ZQHYQN
scraping B00QSR2ABQ
this asin: B00QSR2ABQ has problems on title
scraping B01MG5J1LJ
scraping B01EYT1URO
Expecting value: line 1 column 1 (char 0)
B073J558DB
scraping B00S3FEOAG
scraping B01B2MSEG6
scraping B00KYUXIOG
scraping B00TXR59J8
scraping B01LQV1YNS
this asin:

In [ ]:
onto = get_ontology("http://test.org/onto.owl")
dict_list = json.load(open('tmp.json', 'r'))

In [ ]:
for i in onto.properties():
    print(i)

In [5]:
# special properties: classes, brands, sellers
feature_lists = ['features', 'authors', 'reviewTexts', 'languages', 'creators', 'editorial_reviews']
special_features = ['classes', 'brand', 'sold_by', 'title']
feature_dict = {}

"""
for elem in dict_list:
    for key in elem:
        if key not in feature_lists:
            New_Class = types.new_class(key, )
"""

'\nfor elem in dict_list:\n    for key in elem:\n        if key not in feature_lists:\n            New_Class = types.new_class(key, )\n'

In [6]:
# CLASSES
for i in dict_list:
    with onto:
        if "classes" in i:
            prevclass = Thing
            for itemclass in i["classes"]:
                if itemclass not in [j._name for j in onto.classes()]:
                    prevclass = types.new_class(itemclass, (prevclass,))

In [7]:
# BRANDS
with onto:
    if "Brand" not in [j._name for j in onto.classes()]:
        Brand = types.new_class("Brand", (Thing,))
    else:
        Brand = next(j for j in onto.classes() if j.name == "Brand")
    
for i in dict_list:
    with onto:
        if "brand" in i:
            if i["brand"] not in [j._name for j in onto.classes()] and i["brand"] != None:
                New_Class = types.new_class(i['brand'], (Brand, ))                

In [8]:
# SELLERS
with onto:
    if "Seller" not in [j._name for j in onto.classes()]:
        Seller = types.new_class("Seller", (Thing,))
    else:
        Seller = next(j for j in onto.classes() if j.name == "Seller")
    
for i in dict_list:
    with onto:
        if "sold_by" in i:
            if i["sold_by"] not in [j._name for j in onto.classes()]:
                New_Class = types.new_class(i['sold_by'], (Seller, ))                

In [9]:
# PRODUCT GROUPS
with onto:
    for i in dict_list:
        if "product_group" in i:
            if i["product_group"] not in [j._name for j in onto.classes()]:
                 New_Class = types.new_class(i['product_group'], (Thing, ))

In [10]:
# THINGS
property_dict = {}


for i in dict_list:
    with onto:
        if "title" in i:
            if i["title"] not in [j._name for j in onto.classes()]:
                if "classes" in i and len(i["classes"]) > 0:
                    New_Class1 = types.new_class(i["title"], (next(j for j in onto.classes() if j.name == i["classes"][-1]),))
                elif "product_group" in i and len(i["product_group"]) > 0:
                    New_Class1 = types.new_class(i["title"], (next(j for j in onto.classes() if j.name == i["product_group"]),))
                else:
                    New_Class1 = types.new_class(i["title"], (Thing,))
                
            else: 
                New_Class1 = next(j for j in onto.classes() if j.name == i["title"])

            print("working on " + i["title"])

            # string properties
            for key in i:
                if key not in special_features and (type(i[key]) is str or type(i[key]) is float or type(i[key]) is int)\
                and key!='amazoncom':
                    property_dict[key] = types.new_class(key, (DataProperty, ))
                    try: New_Class1.is_a.append(property_dict[key].value(i[key]))
                    except: print('ERR:' + key + ' ' + i[key])
        

#             if "color" in i:
#                 has_color = types.new_class("has_color", (DataProperty,))
#                 has_color.range = [str]
#                 New_Class1.has_color.append(str(i["color"]))
#                 print(New_Class1.__dict__)

    #         if "price" in i:
    #             has_price = types.new_class("has_price", (DataProperty,))
    #             has_price.range = [float]
    #             New_Class1.has_price.append(float(i["price"]))

    #         """
    #         if "brand" in i:
    #             has_brand = types.new_class("has_brand", (ObjectProperty,))
    #             produces = types.new_class("produces", (ObjectProperty,))
    #             has_brand.inverse_property = produces
    #             New_Class1.has_brand.append(next(j for j in onto.classes() if j.name == i["brand"]))
    #         """

    #         if "sold_by" in i:
    #             sold_by = types.new_class("sold_by", (ObjectProperty,))
    #             sells = types.new_class("sells", (ObjectProperty,))
    #             sold_by.inverse_property = sells
    #             New_Class1.sold_by.append(next(j for j in onto.classes() if j.name == i["sold_by"]))

working on Zebra_Mini_Mechanical_Pencil_TS3_05mm_Silver_(TS3)
working on HTC_Desire_Eye_E1_16GB_White/Red_GSM_Unlocked_US_Version_(13MP_Front_and_Rear_Camera)
working on PopSockets:_Expanding_Stand_and_Grip_for_Smartphones_and_Tablets__Celebration
working on HTC_Desire_610_8GB_Unlocked_GSM_4G_LTE_QuadCore_Android_44_Smartphone__Black_(No_Warranty)
working on Jerrybox_360_Degree_Rotation_Makeup_Organizer_Adjustable_MultiFunction_Cosmetic_Storage_Box_Large_Capacity_7_Layers_Fits_Toner_Creams_Makeup_Brushes_Lipsticks_and_More_Black
working on Samsung_Galaxy_J7_Prime_(32GB)_G610F/DS__55_Dual_SIM_Unlocked_Phone_with_Finger_Print_Sensor_(White_Gold)
working on Hawaiian_Tropic_Sunscreen_Silk_Hydration_Moisturizing_Broad_Spectrum_Sun_Care_Sunscreen_Lotion__SPF_30_6_Ounce
working on Starkist_Tuna_Creations_Variety_Bundle_Single_Serve_26Ounce_Pouch_(Pack_of_5)_includes_1Pouch_Sweet_and_Spicy_plus_1Pouch_Hickory_Smoked_plus_1Pouch_Herb_and_Garlic_plus_1Pouch_Ranch_plus_1Pouch_Lemon_Pepper
working

working on Movie_Night_Hollywood_Themed_Party_Metallic_Movie_Film_Border_Roll_Wall_Decoration_Vinyl_40_Feet_x_18_Inches
working on Zinus_Memory_Foam_12_Inch_Green_Tea_Mattress_Queen
working on Blade_Runner_(1982)__Pack_Collector_(BluRay_plus_Tshirt_plus_8_Post_cards_plus_Poster)_[Bluray]_(European_region_B/2)
working on Cook_N_Home_Nonstick_55_Mini_Size_One_Egg_Fry_Pan_and_Sauce_Pan_1QT_with_Lid_Set_Red
working on ICO_Professional_Whipped_Cream_Dispenser_(1_Pint)
working on Zinus_Deluxe_Faux_Leather_Upholstered_Platform_Bed_with_Wooden_Slats_Queen
working on JBL_GO_Portable_Wireless_Bluetooth_Speaker_W/_A_BuiltIn_StrapHook_(black)
working on VicTsing_Bluetooth_FM_Transmitter_Radio_Car_Kit_Adapter_With_144_Inch_Display_5V_21A_USB_Car_Charger_Support_Micro_SD_Card_and_USB_Flash_DriveBlack
working on Letv_Bluetooth_40_10W_Output_Portable_Wireless_Speaker_(Black)
working on Kitchen_sink_caddy_sponge_holder_scratcher_holder_cleaning_brush_holder_sink_organizer_(green)
working on Samsung_Gal

working on Mpow_Car_Phone_MountWashable_Strong_Sticky_Gel_Pad_with_OneTouch_Design_Dashboard_Car_Phone_Holder_for_iPhone_8/8Plus/7/7Plus/6s/6Plus/5S_Galaxy_S5/S6/S7/S8_Google_Nexus_LG_Huawei_and_More
working on 180PCS_IFfree_Eraser_Caps_Assorted_ColorsFun_Color_for_Fun_Learning_NEWAssorted_ColorsRed_Yellow_Green_Blue_Purple_Orange
working on Rainygo_Travel_Folding_Umbrella_210T_10_Ribs_Strong_Windproof_Compact_Light_Weight_Auto_Open_Close
working on DOSS_SoundBox_Bluetooth_Speaker_Portable_Wireless_Bluetooth_40_Touch_Speakers_with_12W_HD_Sound_and_Bold_Bass_Handsfree_12H_playtime_for_Echo_Dot_iPhone_iPad_Samsung_tablet_Gift_ideas[Blue]
working on Automatic_Compact_Travel_Umbrella_with_Reverse_and_Safe_Lock_Design_HQdeal_Teflon_210T_Auto_Open_Close_Folding_Strong_Windproof_Umbrella_Black
working on The_Art_and_Soul_of_Blade_Runner_2049__Visual_Art_Book_Deluxe_Edition
working on LG_G3_D850_32GB_Unlocked_GSM_4G_LTE_QuadHD_Smartphone_w/_13MP_Camera_(Certified_Refurbished)
working on Creati

working on Universal_Magnetic_Car_Mount__For_any_Phone_GPS_or_Light_Tablet__Stylish_Black_Chrome_OneHand_and_OneSec_Dash_Holder_plus100_to_Safeness_and_Comfort
working on Bleiou_8PCS_Multifunctional_Adhesive_Car_Charger_Line_Clasp_Clamp_Headphone/USB_Cable_Car_Clip_Interior_Accessories
working on Fisher_Price_My_First_Books_Set_of_4_Baby_Toddler_Board_Books_(ABC_Book_Colors_Book_Numbers_Book_Opposites_Book)
working on Bluetooth_SpeakersURPOWER_HiFi_Portable_Wireless_Stereo_Speaker_with_7_LED_Visual_Modes_and_Buildin_Microphone_Support_Handsfree_Function_for_iPhone_7_Plus7SamsungTablets_and_MoreZ3
working on The_Lost_City_of_Z
working on GreenLife_Soft_Grip_8_Ceramic_NonStick_Open_Frypan_Turquoise
working on ASUS_ZenFone_2_ZE551ML_4GB_RAM_/_64GB_ROM_55Inch_2G_/_4G_Dual_SIM_Factory_Unlocked_International_Stock_No_Warranty_(Glacier_Gray)
working on Anker_SoundCore_Bluetooth_Speaker_with_24Hour_Playtime_66Foot_Bluetooth_Range_and_Builtin_Mic_DualDriver_Portable_Wireless_Speaker_with_Superi

working on Apple_iPhone_5_16_GB_Verizon_Black
working on Huawei_Honor_6X_Dual_Camera_Unlocked_Smartphone_32GB_Silver_(US_Warranty)
working on ANRANK_BS63570AK_Super_Bass_Mini_Portable_Bluetooth_Wireless_Speaker_with_LED_and_Buildin_Mic_Support_AUX_TF_iPhone_Samsung_Smartphone_PC_Tablet_(Pink)
working on FIFA_15__Xbox_360
working on Malalas_Magic_Pencil
working on Corel_Painter_Essentials_5
working on URPOWER_500ml_Aromatherapy_Essential_Oil_Diffuser_Humidifier_with_4_Timer_Settings_7_LED_Color_Changing_Lamps_and_Waterless_Auto_Shutoff
working on ISNTREE_Hyaluronic_Acid_Toner_676_fl_oz_Deep_Moisturizing_Soothing_Nourishing
working on The_Very_Best_Of_Cream
working on Just_Give_Me_a_Reason
working on SimpleHouseware_Closet_Underwear_Organizer_Drawer_Divider_4_Set_Gray
working on Casabella_1_Count_Dustpan_and_Brush_Set_Graphite/Orange
working on Bluetooth_speakersPortable_wireless_surround_sound_speakerStereo_speakerThe_pill_car_speaker
working on Im_Not_Dead_(Explicit)
working on Autoark

...

working on Kirkland_Signature_Solid_White_Albacore_Tuna_56_Ounce_Pack_of_2
working on Essy_Beauty_Hyaluronic_Acid_Serum_for_Skin__Advanced_Hydrating_Anti_aging_formula_1_fl_oz
working on LG_G3_D851_32GB_Black_Smartphone_for_TMobile_(Certified_Refurbished)
working on Birds_and_Blooms
working on Raised_Design_10oz_Bee_Mug
working on Missgrace_2PCS_Girl_Exquisite_Gold_Bee_Hairpin_Side_Clip_Hair_Accessorie
working on Liquitex_Professional_Acrylic_Ink_Muted_Colors_Collection_Set
working on Baja_Precious__Rice_Bran_Oil_1_Gallon
working on Amazon_Prime_Video
working on Aspen_Pet_Oval_Cuddler_Pet_Bed_20Inch_by_16Inch_Chocolate_Brown
working on Car_Vacuum_Cleaner_Vacplus_DC_12V_Portable_Handheld_Vacuum_Cleaner_for_Car_with_45KPa_Suction_1_Extra_HEPA_Filter_1_Carrying_Case_164ft_Cable_Electrical_Gauges_Vauum_for_WetandDry_Use_Black
working on Rockford_Fosgate_R14X2_Prime_Full_Range_Coaxial_Speaker__4__Set_of_2
working on IMUSA_USA_IMU91630_Charcoal_Nonstick_Saute_Pan_8Inch_Black
working on Massi

working on FIFA_Street__Playstation_3
working on Feast_from_the_East_All_Natural_Sesame_Salad_Dressing__Large_32_fl_oz
working on Jackey_Awesome_1_X_Peeking_Monster_Scary_Eyes_Car_Decal_/_Sticker_for_Laptop_Ipad_Window_Wall_Car_Truck_Motorcycle_(Theft_Black)
working on Samsung_SmartThings_Smart_Home_Hub
working on Motherlove_Nipple_Cream_Certified_Organic_Salve_for_Sore_Cracked_Nursing_Nipples_1_Oz
working on AXESS_SPBT1031_Portable_Bluetooth_Indoor/Outdoor_21_HiFi_Cylinder_Loud_Speaker_with_BuiltIn_3_Sub_and_SD_Card_USB_AUX_Inputs_in_Red
working on Argireline_Matrixyl_3000_Peptide_cream_Hyaluronic_Acid_Wrinkle_Collagen_C_serum_by_Abcstore99
working on VTech_Musical_Rhymes_Book_(Frustration_Free_Packaging)
working on Window_Alert_Leaf_Medley_Decal_5Pack
working on Signed_Sealed_Delivered:_Lost_Without_You
working on Audubon_Singing_Bird_Clock__13_Green
working on McCormick_Salad_Toppins_Crunchy_and_Flavorful_375oz_Bottle_(Pack_of_3)
working on Tfal_B165SI_Initiatives_Nonstick_Inside_an

working on LOHAS_Smart_LED_Bulb_WiFi_Light_Multicolored_LED_Bulbs_A19_LED_Dimmable_60W_Equivalent(9W)_Smartphone_Controlled_Daylight_and_Night_Light_Home_Lighting_Works_with_Amazon_Alexa(1_Pack)
working on Touch_of_Nature_20556_Baltimore_Oriole_Bird_4Inch
working on Football_tournament_2014
working on StarKist_Tuna_Creations_Sweet_and_Spicy_26_Ounce_(Pack_of_12)
working on BESSKY_2PCS_Style_Girl_Exquisite_Gold_Bee_Hairpin_Side_Clip_Hair_Accessories
working on Homak_HA01012001_12Drawer_Parts_Organizer_Black
working on JVC_CSJ620_300W_65_CS_Series_2Way_Coaxial_Car_Speakers_Set_of_2
working on LifeTek_Travel_Umbrella_Windproof_Compact_Automatic_Open_Close_Small_Folding_Teflon_Repellent_Canopy_Umbrellas_fits_Golf_Purse_Backpack_Wind_Resistant_for_Men_and_Women_Traveler_FX1_42_inch_Black
working on Cook_N_Home_12Inch_Frying_Pan_with_NonStick_Coating_Induction_Compatible_Bottom_Large_Green
working on Cant_Take_Me_Home
working on 10_Stone_Earth_Frying_Pan_by_Ozeri_with_100percent_APEO_and_PFO

working on TaoTronics_Dash_Cam_2K_30fps_HD_Super_Night_Vision_Car_Camera_DVR_2_LCD_160_Wide_Angle_Dashboard_Recorder_Sony_GSensor_Loop_Recording_HDR_Free_32GB_SD_Card
working on Wooden_Folding_Book_Light_Magicfly_USB_Rechargable_Book_Shaped_Light_Warm_White_Led_Desk_Table_Lamp_for_Decor_Magnetic_Design_Creative_gift
working on DROK_Mini_5W_Stereo_Audio_Speaker_2_inch_Hifi_Speaker_4_Ohm_Fullrange_DIY_Loudspeaker_Woofer_Active_Portable_Speaker_Indoor_Outdoor_Usage_Car_Auto_Motor_Motorcycle
working on Amscan_Hollywood_Movie_Themed_Party_Decorating_Kit_(10_Piece)_Black/Gold/Silver_155_x_108
working on Greenco_6_Piece_Refrigerator_and_Freezer_Stackable_Storage_Organizer_Bins_with_Handles_Clear
working on Cell_Phone_Holder_for_Car_Car_Phone_Mounts_for_iPhone_7_Plus_Dashboard_GPS_Holder_Mounting_in_Vehicle_for_Samsung_Galaxy_S8_and_other_368_Inch_Universal_Smartphones_and_GPS__Black
working on 5_Pieces_Set_for_Cabinet_Organizer
working on Watercolor_Paint_Set_by_AEM_Hi_Arts__24_Color_Watercol

working on Mazola_Corn_Oil__16_oz
working on Crayola_Marker_and_Watercolor_Pad_10_x_8_Inches__50_pages
working on Orginal_Oneplus_X_4G_Smartphone_Android_51_Quad_Core_23GHz_3GB_RAMplus16GB_ROM_with_50_inch_Screen_Dual_Sim_Card_International_Version_(White_with_Gold)
working on Slim_Rolling_Bathroom_Organizer__White
working on Inconvenient_Sequel:_Truth_to_Power_An
working on DOSS_SoundBox_Bluetooth_Speaker_Portable_Wireless_Bluetooth_40_Touch_Speakers_with_12W_HD_Sound_and_Bold_Bass_Handsfree_12H_playtime_for_Echo_Dot_iPhone_iPad_Samsung_tablet_Gift_ideas[White]
working on Magnetic_Gun_Mount_and_Holster_For_Vehicle_And_Home__HQ_Rubber_Coated_35_Lbs_Rated__Firearm_Accessory_Concealed_Holder_For_Handgun_Rifle_Shotgun_Pistol_Revolver_Truck_Car_Wall_Vault_Desk
working on Prevue_Hendryx_62806_Naturals_Rope_Ladder_Bird_Toy
working on BLU_VIVO_XL2__55_4G_LTE_GSM_Unlocked_Smartphone___32GBplus3GB_RAM_Gold
working on KandN_KN204_Motorcycle/Powersports_High_Performance_Oil_Filter
working on FIFA

In [ ]:
for i in dict_list:
    pprint(i)

In [ ]:
sync_reasoner()
for i in onto.classes():
    print(i.has_color)

In [ ]:
onto.OnePlus_One_A0001_4G_LTE_3GBRAM_64GBROM_55_Inch_International_Version__No_Warranty__Sandstone_Black.has_color

In [ ]:
graph = default_world.as_rdflib_graph()
r = list(graph.query("""SELECT * WHERE { ?a ?b ?c . }"""))
#r = list(graph.query("""SELECT * WHERE { ?a ?b <http://test.org/onto.owl#has_color> }"""))

pprint(r)


#onto.search( has_color = "*")

In [11]:
onto.save("saved_ontology.rdf")

In [ ]:
camera_set = set()
color_set = set()
for i in dict_list:
    if 'camera description' in i: camera_set.add(i['camera description'])
    if 'color' in i: color_set.add(i['color'])
    
pprint(color_set)

In [ ]:
Phone = types.new_class("Phone", (Thing,), kwds = { "ontology" : onto })
Camera = types.new_class("Camera", (Thing,), kwds = { "ontology" : onto })
Color = types.new_class("Color", (Thing,), kwds = { "ontology" : onto })
has_camera = types.new_class("has_camera", (Property,), kwds = { "ontology" : onto })
has_color = types.new_class("has_color", (Property,), kwds = { "ontology" : onto })
ANNOTATIONS[has_camera]["python_name"] = "has_camera"
ANNOTATIONS[has_color]["python_name"] = "has_color"

In [ ]:
for i in color_set:
    Color(i)

In [ ]:
for i in onto.Color._direct_instances:
    print(vars(i))

In [ ]:
for i in camera_set:
    Camera(i)

In [ ]:
for i in dict_list:
    which_color = ""
    if 'color' not in i: continue
    for j in onto.Color._direct_instances:
        if j._name == i['color']:
            which_color = j
    New_Class = types.new_class(i["asin"][:-5], (Phone,), kwds = { "ontology" : onto })
    New_Class.equivalent_to = [Phone & restriction(has_color, VALUE, j)]

In [ ]:
for i in dict_list:
    

In [ ]:
pprint(onto.instances)

In [ ]:
class Phone(Thing):
    ontology = onto
    
class Camera(Thing):
    ontology = onto
    
class has_camera(Property):
    ontology = onto
    domain = [Phone]
    range = [Camera]
ANNOTATIONS[has_camera]["python_name"] = "has_camera"

camera_8mp = Camera("camera_8mp")
camera_16mp = Camera("camera_16mp")

class galaxy_s8 (Phone):
    ontology = onto
    equivalent_to = [Phone & restriction(has_camera, VALUE, camera_16mp)]
    
    def print_name(self):
        print("hey there")

In [ ]:
pprint(vars(onto.galaxy_s8))

In [ ]:
query = Phone(has_color = [onto.black])
query.closed_world()
onto.sync_reasoner()

In [ ]:
print(vars(query))

In [ ]:
for ontology in onto.indirectly_imported_ontologies(): configure_editobj_from_ontology(ontology)

editobj3.GUI = "Qt" # "Qt" or "HTML"

if editobj3.GUI == "Qt": # Required by Qt
    import sys, PyQt5.QtWidgets as qtwidgets
    if qtwidgets.QApplication.startingUp(): qtwidgets.app = qtwidgets.QApplication(sys.argv)

editor = OntologyInstanceEditor()
editor.set_ontology(onto, edited_classes = onto.classes)
editor.main()